cold-user에게 정보를 받는 법 고민해보기

In [12]:
import sys
import os

# 주피터 노트북 환경에서 __file__이 없으므로, 현재 워킹 디렉토리 기준으로 설정
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import random
from data_scraping.common.data_loader import load_ratings_data

df_ratings = load_ratings_data()
lot_rated_movie_ids = set(df_ratings.value_counts(subset='movie_id').sort_values(ascending=False).values[:100])

# 여기 있는 영화를 보셨나요?
selected_movie_ids = set(random.sample(lot_rated_movie_ids, 10))
lot_rated_movie_ids = lot_rated_movie_ids - selected_movie_ids

# 버튼 다시 누르면
selected_movie_ids = set(random.sample(lot_rated_movie_ids, 10))

/var/folders/bw/_nzx77m90994jql91xkcq2kc0000gn/T/ipykernel_87052/3231592725.py:16: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  selected_movie_ids = set(random.sample(lot_rated_movie_ids, 10))
/var/folders/bw/_nzx77m90994jql91xkcq2kc0000gn/T/ipykernel_87052/3231592725.py:20: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  selected_movie_ids = set(random.sample(lot_rated_movie_ids, 10))


# 수집한 거 불러오기

In [6]:
from modeling.utils.data_integration import DataIntegrator

# Firebase 초기화
from user_system.firebase_config import setup_firebase_config
import streamlit as st

# Firebase 설정
firebase_available = setup_firebase_config()
print(f"Firebase 초기화 상태: {firebase_available}")

Firebase 초기화 상태: True


In [7]:
# Firebase 초기화 확인
if firebase_available:
    print("✅ Firebase가 성공적으로 초기화되었습니다.")
    
    # DataIntegrator 초기화
    integrator = DataIntegrator()
    
    # Firebase 데이터 로드 테스트
    print("Firebase 데이터를 로드하는 중...")
    firebase_data = integrator.load_firebase_data()
    print(f"Firebase 데이터: {len(firebase_data)}개 평점")
    
    if not firebase_data.empty:
        print("Firebase 데이터 샘플:")
        print(firebase_data.head())
    else:
        print("⚠️ Firebase에서 데이터를 찾을 수 없습니다.")
else:
    print("❌ Firebase 초기화에 실패했습니다.")
    print("Firebase 설정 파일을 확인해주세요.")


✅ Firebase가 성공적으로 초기화되었습니다.
Firebase 데이터를 로드하는 중...
Firebase 데이터: 12개 평점
Firebase 데이터 샘플:
                        user_id movie_id  rating
0  SOMV3U3OEWT7LG9t2VMeNXKQchd2  m5xMKrW     1.0
1  SOMV3U3OEWT7LG9t2VMeNXKQchd2  mJ5x8x5     5.0
2  SOMV3U3OEWT7LG9t2VMeNXKQchd2  mJ5xvEO     3.5
3  SOMV3U3OEWT7LG9t2VMeNXKQchd2  mJOV1RW     4.0
4  SOMV3U3OEWT7LG9t2VMeNXKQchd2  mOVgp1d     5.0


In [8]:
# 데이터 통합 테스트
if firebase_available:
    print("=== 데이터 통합 테스트 ===")
    
    # 1. 기존 데이터 로드
    print("1. 기존 데이터를 로드하는 중...")
    original_data = integrator.load_original_data()
    print(f"기존 데이터: {len(original_data)}개 평점")
    
    # 2. Firebase 데이터 로드
    print("2. Firebase 데이터를 로드하는 중...")
    firebase_data = integrator.load_firebase_data()
    print(f"Firebase 데이터: {len(firebase_data)}개 평점")
    
    # 3. 데이터 통합
    if not original_data.empty or not firebase_data.empty:
        print("3. 데이터를 통합하는 중...")
        integrated_data = integrator.integrate_data(original_data, firebase_data)
        print(f"통합된 데이터: {len(integrated_data)}개 평점")
        
        if not integrated_data.empty:
            # 통계 정보
            stats = integrator.get_data_statistics(integrated_data)
            print("\n=== 통합된 데이터 통계 ===")
            print(f"총 평점 수: {stats['total_ratings']:,}")
            print(f"고유 사용자: {stats['unique_users']:,}")
            print(f"고유 영화: {stats['unique_movies']:,}")
            print(f"평균 평점: {stats['avg_rating']:.2f}")
            
            # 데이터 샘플
            print("\n=== 통합된 데이터 샘플 ===")
            print(integrated_data.head())
        else:
            print("⚠️ 통합된 데이터가 비어있습니다.")
    else:
        print("⚠️ 통합할 데이터가 없습니다.")
else:
    print("Firebase가 초기화되지 않아 데이터 통합을 진행할 수 없습니다.")


=== 데이터 통합 테스트 ===
1. 기존 데이터를 로드하는 중...
기존 데이터: 3361101개 평점
2. Firebase 데이터를 로드하는 중...
Firebase 데이터: 12개 평점
3. 데이터를 통합하는 중...
통합된 데이터: 3361113개 평점

=== 통합된 데이터 통계 ===
총 평점 수: 3,361,113
고유 사용자: 4,458
고유 영화: 49,856
평균 평점: 3.32

=== 통합된 데이터 샘플 ===
         user_id movie_id   movie_title  rating
0  ZBm5R0Mj7qd46  m5XKVDL         양자물리학     3.0
1  ZBm5R0Mj7qd46  m5Nn2GE       영화로운 작음     3.5
2  ZBm5R0Mj7qd46  mWvq0vZ           내 방     2.5
3  ZBm5R0Mj7qd46  mdKpyBk      성적표의 김민영     3.0
4  ZBm5R0Mj7qd46  mOopbwm  킹덤4: 대장군의 귀환     3.0


In [15]:
from modeling.models.svd import SVDRecommenderPipeline, ModelConfig
from pathlib import Path

config = ModelConfig.from_yaml()

pipeline = SVDRecommenderPipeline(config)
metrics = pipeline.run_full_pipeline()

# 모델 저장
model_path = '/Users/user/Desktop/movie_recommendation/modeling/models/pkls/trained_svd_pipeline.pkl'
pipeline.save_model(str(model_path))

loaded_pipeline = SVDRecommenderPipeline.load_model(str(model_path))


[필터링 전] 사용자 수: 4,457명, 영화 수: 49,856개
[필터 적용] min_user_ratings: 30, min_movie_ratings: 10
조건 통과 사용자 수: 4,343명, 조건 통과 영화 수: 15,770개
[필터링 후] 사용자 수: 4,343명, 영화 수: 15,770개
필터링된 평점 수: 3,269,922개 (제거: 91,179개)
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
RMSE: 0.7131
MAE:  0.5426
RMSE: 0.6986
MAE:  0.5323


In [ ]:
from data_scraping.common.data_loader import load_movie_data, load_ratings_data

df_movies = load_movie_data()

In [22]:
df_ratings = load_ratings_data()

In [33]:
df_ratings[df_ratings['movie_id']=='mObGGj7']

,user_id,movie_id,movie_title,rating
1106,ZBm5R0Mj7qd46,mObGGj7,베테랑,3.5
4281,24BqEn96mvrjn,mObGGj7,베테랑,2.0
4907,JZ4vB0GPl1xRX,mObGGj7,베테랑,3.0
5028,VRZv4L8MMbvr6,mObGGj7,베테랑,4.0
5739,nkPvrXRBDxgar,mObGGj7,베테랑,4.0
...,...,...,...,...
3358621,3BnvwRKV9qPAY,mObGGj7,베테랑,3.0
3358954,4WLxZzzmZxroA,mObGGj7,베테랑,4.0
3359661,3BnvwRwpZqPAY,mObGGj7,베테랑,4.0
3360174,7gdxdy2gwqGYJ,mObGGj7,베테랑,4.5


In [38]:
'SOMV3U3OEWT7LG9t2VMeNXKQchd2' in loaded_pipeline.df_filtered['user_id'].unique()

False

In [63]:
integrated_data[integrated_data['user_id'] == 'SOMV3U3OEWT7LG9t2VMeNXKQchd2']

,user_id,movie_id,movie_title,rating
3361101,SOMV3U3OEWT7LG9t2VMeNXKQchd2,m5xMKrW,NaN,1.0
3361102,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mJ5x8x5,NaN,5.0
3361103,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mJ5xvEO,NaN,3.5
3361104,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mJOV1RW,NaN,4.0
3361105,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mOVgp1d,NaN,5.0
3361106,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mObGGj7,NaN,4.0
3361107,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mWpkJw5,NaN,4.5
3361108,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mWqJnGr,NaN,4.0
3361109,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mWw8zEO,NaN,1.0
3361110,SOMV3U3OEWT7LG9t2VMeNXKQchd2,mY5QnJW,NaN,4.0


In [ ]:
from modeling.utils.data import filter_by_min_counts

filter_by_min_counts(integrated_data)


[필터링 전] 사용자 수: 4,458명, 영화 수: 49,856개
[필터 적용] min_user_ratings: 30, min_movie_ratings: 10
조건 통과 사용자 수: 4,343명, 조건 통과 영화 수: 15,770개
[필터링 후] 사용자 수: 4,343명, 영화 수: 15,770개
필터링된 평점 수: 3,269,922개 (제거: 91,191개)


,user_id,movie_id,movie_title,rating
0,ZBm5R0Mj7qd46,m5XKVDL,양자물리학,3.0
3,ZBm5R0Mj7qd46,mdKpyBk,성적표의 김민영,3.0
4,ZBm5R0Mj7qd46,mOopbwm,킹덤4: 대장군의 귀환,3.0
5,ZBm5R0Mj7qd46,mWya9Bx,한 채,3.0
6,ZBm5R0Mj7qd46,m5nXwBD,킹덤 3: 운명의 불꽃,3.0
...,...,...,...,...
3361096,ld0q0rrGR56Xn,mmOb3kd,해리포터와 마법사의 돌,5.0
3361097,ld0q0rrGR56Xn,mWyQz6W,엣지 오브 투모로우,4.0
3361098,ld0q0rrGR56Xn,mAd7zBO,제5원소,4.0
3361099,ld0q0rrGR56Xn,mM5rmEd,해리포터와 불의 잔,4.5


In [52]:
# 'SOMV3U3OEWT7LG9t2VMeNXKQchd2' 사용자의 평점 개수 추출
user_id = 'SOMV3U3OEWT7LG9t2VMeNXKQchd2'
user_rating_count = integrated_data[integrated_data['user_id'] == user_id].shape[0]
print(f"{user_id} 평점 개수:", user_rating_count)

SOMV3U3OEWT7LG9t2VMeNXKQchd2 평점 개수: 12


In [17]:
top_watched, recommendations = loaded_pipeline.recommend_for_user(
    'SOMV3U3OEWT7LG9t2VMeNXKQchd2', df_movies, 10
)

ValueError: 사용자 ID 'SOMV3U3OEWT7LG9t2VMeNXKQchd2'를 찾을 수 없습니다.